In [1]:
get_ipython().ast_node_interactivity = 'all'

In [1]:
# Full dashboard cell — single Output guaranteed (HTML placeholders used for text output)
import requests
import math
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, HTML

# API setup
API_base = "https://statsapi.mlb.com/api/v1"
Teams_URL = f"{API_base}/teams?sportId=1"
Roster_URL = f"{API_base}/teams/{{team_id}}/roster"
Player_Stats_URL = f"{API_base}/people/{{player_id}}/stats"

sess = requests.Session()
sess.headers.update({ "User-Agent": "mlb-jupyter-dashboard/1.0" })

# caches / globals
team_list = []
team_name_to_id = {}
team_roster_cache = {}   # team_name.lower() -> newline roster string
player_name_cache = {}
player_dictionaries = {}  # roster cache by team id

# retrieve teams once
try:
    r = sess.get(Teams_URL, timeout = 10)
    r.raise_for_status()
    team_list = r.json().get("teams", []) or []
    for t in team_list:
        nm = t.get("name", "")
        if nm:
            team_name_to_id[nm.lower()] = t.get("id")
except Exception as e:
    # not fatal; UI will still show a message
    team_list = []

# image map for widget 1 (team logos)
image_map = {
    "athletics": "https://www.mlbstatic.com/team-logos/team-cap-on-light/133.svg",
    "pirates": "https://www.mlbstatic.com/team-logos/team-cap-on-light/134.svg",
    "padres": "https://www.mlbstatic.com/team-logos/team-cap-on-light/135.svg",
    "mariners": "https://www.mlbstatic.com/team-logos/team-primary-on-light/136.svg",
    "giants": "https://www.mlbstatic.com/team-logos/team-cap-on-light/137.svg",
    "cardinals": "https://www.mlbstatic.com/team-logos/team-cap-on-light/138.svg",
    "rays": "https://www.mlbstatic.com/team-logos/team-cap-on-light/139.svg",
    "rangers": "https://www.mlbstatic.com/team-logos/team-cap-on-light/140.svg",
    "blue jays": "https://www.mlbstatic.com/team-logos/team-cap-on-light/141.svg",
    "twins": "https://www.mlbstatic.com/team-logos/team-cap-on-light/142.svg",
    "phillies": "https://www.mlbstatic.com/team-logos/team-cap-on-light/143.svg",
    "braves": "https://www.mlbstatic.com/team-logos/team-cap-on-light/144.svg",
    "white sox": "https://www.mlbstatic.com/team-logos/team-cap-on-light/145.svg",
    "marlins": "https://www.mlbstatic.com/team-logos/team-cap-on-light/146.svg",
    "yankees": "https://www.mlbstatic.com/team-logos/team-cap-on-light/147.svg",
    "brewers": "https://www.mlbstatic.com/team-logos/team-cap-on-light/158.svg",
    "angels": "https://www.mlbstatic.com/team-logos/team-cap-on-light/108.svg",
    "diamondbacks": "https://www.mlbstatic.com/team-logos/team-cap-on-light/109.svg",
    "orioles": "https://www.mlbstatic.com/team-logos/team-cap-on-light/110.svg",
    "red sox": "https://www.mlbstatic.com/team-logos/team-cap-on-light/111.svg",
    "cubs": "https://www.mlbstatic.com/team-logos/team-cap-on-light/112.svg",
    "reds": "https://www.mlbstatic.com/team-logos/team-cap-on-light/113.svg",
    "guardians": "https://www.mlbstatic.com/team-logos/team-cap-on-light/114.svg",
    "rockies": "https://www.mlbstatic.com/team-logos/team-cap-on-light/115.svg",
    "tigers": "https://www.mlbstatic.com/team-logos/team-cap-on-light/116.svg",
    "astros": "https://www.mlbstatic.com/team-logos/team-primary-on-light/117.svg",
    "royals": "https://www.mlbstatic.com/team-logos/team-cap-on-light/118.svg",
    "dodgers": "https://www.mlbstatic.com/team-logos/team-cap-on-light/119.svg",
    "nationals": "https://www.mlbstatic.com/team-logos/team-cap-on-light/120.svg",
    "mets": "https://www.mlbstatic.com/team-logos/team-cap-on-light/121.svg"
}

TWO_WAY_PLAYER_IDS = { 660271 }  # Shohei Ohtani ID

# helpers
def get_team_roster_text(team_id):
    """Return newline-joined 'Name, Position' roster string for a team (cached)."""
    # attempt to find team name
    t = next((x for x in team_list if x.get("id") == team_id), None)
    team_name = t.get("name") if t else str(team_id)
    key = str(team_name).lower()
    if key in team_roster_cache:
        return team_roster_cache[key]
    # fetch roster if necessary
    try:
        url = Roster_URL.format(team_id = team_id)
        r = sess.get(url, timeout = 10)
        r.raise_for_status()
        roster = r.json().get("roster", []) or []
    except Exception:
        roster = []
    lines = []
    for p in roster:
        nm = p.get("person", {}).get("fullName")
        pos = p.get("position", {}).get("name")
        if nm:
            lines.append(f"{nm}, {pos}")
    txt = "\n".join(lines)
    team_roster_cache[key] = txt
    return txt

def get_player_name(player_id, timeout = 5):
    """Cached player name lookup, falls back to id string on failure."""
    if player_id in player_name_cache:
        return player_name_cache[player_id]
    try:
        url = f"{API_base}/people/{player_id}"
        r = sess.get(url, timeout = timeout)
        r.raise_for_status()
        people = r.json().get("people", [])
        name = people[0].get("fullName") if people else str(player_id)
    except Exception:
        name = str(player_id)
    player_name_cache[player_id] = name
    return name

def get_roster_for_team_id(team_id, timeout = 10):
    """Return parsed roster objects for a team id (cache by team id)."""
    if team_id in player_dictionaries:
        return player_dictionaries[team_id]
    try:
        url = Roster_URL.format(team_id = team_id)
        r = sess.get(url, timeout = timeout)
        r.raise_for_status()
        roster = r.json().get("roster", []) or []
    except Exception:
        roster = []
    player_dictionaries[team_id] = roster
    return roster

def get_player_season_stats(player_id, season, timeout = 10):
    params = { "stats": "season", "season": str(season) }
    url = Player_Stats_URL.format(player_id = player_id)
    r = sess.get(url, params = params, timeout = timeout)
    r.raise_for_status()
    return r.json()

# Widget 1: Team logo search (text) with instruction
instruction1 = widgets.HTML("<b>Enter MLB team name to view their team logo (do not enter the city, just the team name):</b>")
text = widgets.Text(
    placeholder = "e.g. Tigers, Dodgers, Yankees — enter team name only",
    description = "Team:",
)
text.continuous_update = False

# Replace Output widgets used only for simple display with HTML placeholders
out_logo = widgets.HTML(value = "")                # will hold logo HTML or "Team not found."
teams_out = widgets.HTML(value = "")               # (not used heavily, kept as placeholder)
message_out = widgets.HTML(value = "")             # (errors / messages)
main_out = widgets.HTML(value = "<pre></pre>")     # main text area for roster/stats (preformatted)

# Create a single dashboard output that will contain everything
dashboard_out = widgets.Output(layout = {'border': '1px solid #444', 'padding': '6px', 'height': '900px', 'overflow': 'auto'})

# Controls for Widget3
team_dropdown = widgets.Dropdown(
    options = [("(choose a team)", None)] + [(t.get("name", ""), t.get("id")) for t in team_list],
    value = None,
    description = "Team:",
    layout = widgets.Layout(width = "50%")
)

player_dropdown = widgets.Dropdown(
    options = [("(choose a player)", None)],
    description = "Player:",
    layout = widgets.Layout(width = "60%")
)

season_slider = widgets.IntSlider(
    value = 2025,
    min = 2010,
    max = 2025,
    step = 1,
    description = "Season:",
    continuous_update = False,
    layout = widgets.Layout(width = "50%")
)

show_stats_button = widgets.Button(description = "Show Stats", button_style = "primary", disabled = True)
show_roster_button = widgets.Button(description = "Show Roster", button_style = "info")
refresh_teams_button = widgets.Button(description = "Refresh Teams", button_style = "warning")

# Format player label without id (Name — POS)
def _format_player_label(player_dict):
    name = player_dict.get("person", {}).get("fullName", "Unknown")
    pos = player_dict.get("position", {}).get("abbreviation", "")
    return f"{name}{' — ' + pos if pos else ''}"

# Populate player dropdown for a team
def players_on_team(team_id):
    if team_id is None:
        player_dropdown.options = [("(choose a player)", None)]
        player_dropdown.value = None
        return
    try:
        roster = get_roster_for_team_id(team_id)
    except Exception as e:
        message_out.value = f"<div style='color:#900'>Error fetching roster: {e}</div>"
        player_dropdown.options = [("(choose a player)", None)]
        player_dropdown.value = None
        return
    options = [("(choose a player)", None)]
    for p in roster:
        pid = p.get("person", {}).get("id")
        label = _format_player_label(p)
        if pid in TWO_WAY_PLAYER_IDS:
            hitter_label = f"{p.get('person', {}).get('fullName','Unknown')} (H) — {p.get('position',{}).get('abbreviation','')}"
            pitcher_label = f"{p.get('person', {}).get('fullName','Unknown')} (P) — {p.get('position',{}).get('abbreviation','')}"
            options.append((hitter_label, (pid, "hitting")))
            options.append((pitcher_label, (pid, "pitching")))
        else:
            options.append((label, pid))
    player_dropdown.options = options
    player_dropdown.value = None
    message_out.value = ""  # clear any previous message

# Display roster with team name (ex. "Detroit Tigers Roster")
def display_roster(team_id):
    if team_id is None:
        main_out.value = "<pre>Pick a team to see its roster.</pre>"
        return
    roster = get_roster_for_team_id(team_id)
    team_name = next((t.get("name") for t in team_list if t.get("id") == team_id), str(team_id))
    if not roster:
        main_out.value = f"<pre>{team_name} Roster\n\n(no roster entries)</pre>"
        return
    lines = [f"{team_name} Roster", ""]
    for p in roster:
        name = p.get("person", {}).get("fullName", "Unknown")
        pos = p.get("position", {}).get("name", "")
        lines.append(f"{name}, {pos}")
    main_out.value = "<pre>" + "\n".join(lines) + "</pre>"

# Display player stats — header first "YEAR stats — Player Name" then season/team line
def display_player_stats(player_id, season):
    if not player_id:
        main_out.value = "<pre>Pick a player first.</pre>"
        return
    forced_role = None
    if isinstance(player_id, tuple) and len(player_id) == 2:
        pid, forced_role = player_id
        player_id = pid
    else:
        forced_role = None

    player_name = get_player_name(player_id)

    # Get payload (explicit hitting group when forced - Shohei Ohtani)
    try:
        if forced_role == "hitting":
            url = f"{API_base}/people/{player_id}/stats"
            params = { "stats": "season", "season": str(season), "group": "hitting" }
            r = sess.get(url, params = params, timeout = 10)
            r.raise_for_status()
            payload = r.json()
        else:
            payload = get_player_season_stats(player_id, season)
    except Exception as e:
        main_out.value = f"<pre>Error retrieving player stats: {e}</pre>"
        return

    stats_blocks = payload.get("stats", [])

    # collect all team names seen across splits
    overall_team_names = []
    for b in stats_blocks:
        for s in b.get("splits", []):
            t = s.get("team")
            if isinstance(t, dict):
                nm = t.get("name")
                if nm and nm not in overall_team_names:
                    overall_team_names.append(nm)

    # header lines
    header_lines = [f"{season} stats — {player_name}"]
    if overall_team_names:
        team_part = " and ".join(overall_team_names)
        header_lines.append(f"season = {season} | team = {team_part}")
    else:
        header_lines.append(f"season = {season}")
    header_lines.append("")  # blank line

    # existing display logic (decide hitting vs pitching, choose split, print preferred keys)
    PITCH_PRIMARY = { "ip", "era", "whip", "sv", "er", "so", "fip", "inningsPitched" }
    HIT_PRIMARY = { "ab", "avg", "ops", "obp", "slg", "h", "homeRuns", "rbi", "atBats" }

    def has_pitch_keys(keys):
        return bool(keys & PITCH_PRIMARY)

    def has_hit_keys(keys):
        return bool(keys & HIT_PRIMARY)

    def classify_block(keys, forced_role = None):
        hit = has_hit_keys(keys)
        pitch = has_pitch_keys(keys)
        if forced_role == "hitting":
            return True, False
        if forced_role == "pitching":
            return False, True
        if hit and pitch:
            return True, True
        return hit, pitch

    # detect if player has pitching blocks
    global_is_pitcher = False
    for b in stats_blocks:
        for s in b.get("splits", []):
            keys = set(s.get("stat", {}).keys())
            if has_pitch_keys(keys):
                global_is_pitcher = True
                break
        if global_is_pitcher:
            break

    has_pitch_blocks = any(
        has_pitch_keys(set(s.get("stat", {}).keys()))
        for b in stats_blocks
        for s in b.get("splits", [])
    )
    has_hit_blocks = any(
        has_hit_keys(set(s.get("stat", {}).keys()))
        for b in stats_blocks
        for s in b.get("splits", [])
    )

    if forced_role in ("hitting", "pitching"):
        effective_stat_type = forced_role
    else:
        if player_id in TWO_WAY_PLAYER_IDS and has_hit_blocks and has_pitch_blocks:
            effective_stat_type = "both"
        else:
            effective_stat_type = "pitching" if global_is_pitcher else "hitting"

    if not stats_blocks:
        main_out.value = "<pre>No stats returned for that season.</pre>"
        return

    seen = set()

    #Convert innings pitched (ip) into float innings
    def _ip_to_float(ip):
        try:
            parts = str(ip).split('.')
            if len(parts) == 1:
                return float(parts[0])
            whole = int(parts[0])
            frac = int(parts[1])
            return whole + (frac / 3.0)
        except Exception:
            try:
                return float(ip)
            except Exception:
                return 0.0

    #Choose pitcher vs hitter stats by player position
    def _choose_overall_split(splits):
        if not splits:
            return None
        for s in splits:
            if s.get('split') in ('overall', 'season', 'total'):
                return s
            t = s.get('type')
            if isinstance(t, dict) and 'displayName' in t and str(t['displayName']).lower() in ('season', 'total', 'overview'):
                return s
        best = None
        best_score = -1
        for s in splits:
            stat = s.get('stat', {}) or {}
            ab = int(stat.get('ab') or 0)
            ip = _ip_to_float(stat.get('ip') or 0)
            score = ab + int(ip * 1000)
            if score > best_score:
                best_score = score
                best = s
        return best

    body_lines = []
    for block in stats_blocks:
        disp = block.get("type", {}).get("displayName", "")
        splits = block.get("splits", [])
        if not splits:
            continue
        first_stat = splits[0].get("stat", {}) if splits else {}
        sig = (disp, tuple(sorted(first_stat.keys())))
        if sig in seen:
            continue
        seen.add(sig)

        keys = set().union(*(s.get("stat", {}).keys() for s in splits))

        if player_id in TWO_WAY_PLAYER_IDS and forced_role == "hitting":
            block_is_hitting, block_is_pitching = True, False
        elif player_id in TWO_WAY_PLAYER_IDS and forced_role == "pitching":
            block_is_hitting, block_is_pitching = False, True
        else:
            block_is_hitting, block_is_pitching = classify_block(keys, forced_role = forced_role)

        if not block_is_hitting and not block_is_pitching:
            block_is_pitching = has_pitch_keys(keys)
            block_is_hitting = has_hit_keys(keys)

        if forced_role == "hitting" and not block_is_hitting:
            continue
        if forced_role == "pitching" and not block_is_pitching:
            continue
        if effective_stat_type == "hitting" and not block_is_hitting:
            continue
        if effective_stat_type == "pitching" and not block_is_pitching:
            continue

        # prefer splits that match role
        filtered_splits = splits
        if forced_role == "hitting":
            filtered_splits = [s for s in splits if has_hit_keys(set(s.get("stat", {}).keys()))] or splits
        elif forced_role == "pitching":
            filtered_splits = [s for s in splits if has_pitch_keys(set(s.get("stat", {}).keys()))] or splits
        else:
            if effective_stat_type == "hitting":
                filtered_splits = [s for s in splits if has_hit_keys(set(s.get("stat", {}).keys()))] or splits
            elif effective_stat_type == "pitching":
                filtered_splits = [s for s in splits if has_pitch_keys(set(s.get("stat", {}).keys()))] or splits
            else:
                filtered_splits = splits

        chosen = _choose_overall_split(filtered_splits) or _choose_overall_split(splits)
        if not chosen:
            continue

        # header for block
        body_lines.append(f"{block.get('type', {}).get('displayName', 'Stats')}")
        if chosen.get('split') in ('overall', 'season', 'total'):
            body_lines.append(" Overall season totals ")

        stat = chosen.get('stat', {}) or {}

        if not stat.get("age"):
            for s_age_search in splits:
                other_age = (s_age_search.get("stat") or {}).get("age")
                if other_age:
                    stat["age"] = other_age
                    break
        
        HIT_PREFERRED = [
            ("age", ["age"]),
            ("gamesPlayed", ["gamesPlayed"]),
            ("ab", ["ab", "atBats"]),
            ("h", ["h", "hits"]),
            ("avg", ["avg"]),
            ("homeRuns", ["homeRuns", "hr"]),
            ("rbi", ["rbi"]),
            ("obp", ["obp"]),
            ("slg", ["slg"]),
            ("ops", ["ops"]),
            ("k", ["so", "strikeOuts", "k"]),
            ("bb", ["bb", "baseOnBalls"]),
            ("stolenBases", ["stolenBases", "sb"]),
            ("caughtStealing", ["caughtStealing"]),
            ("r", ["r", "runs"]),
            ("doubles", ["doubles", "twoB"]),
            ("triples", ["triples", "threeB"]),
        ]
        PITCH_PREFERRED = [
            ("age", ["age"]),
            ("ip", ["ip", "inningsPitched"]),
            ("era", ["era"]),
            ("whip", ["whip"]),
            ("k", ["so", "strikeOuts", "k"]),
            ("bb", ["bb", "baseOnBalls"])
        ]
        key_labels = {
            "age": "AGE",
            "gamesPlayed": "G",
            "ab": "AB",
            "atBats": "AB",
            "h": "H",
            "hits": "H",
            "avg": "AVG",
            "homeRuns": "HR",
            "hr": "HR",
            "rbi": "RBI",
            "obp": "OBP",
            "slg": "SLG",
            "ops": "OPS",
            "so": "K",
            "strikeOuts": "K",
            "k": "K",
            "bb": "BB",
            "baseOnBalls": "BB",
            "stolenBases": "SB",
            "sb": "SB",
            "caughtStealing": "CS",
            "r": "R",
            "runs": "R",
            "doubles": "2B",
            "twoB": "2B",
            "triples": "3B",
            "threeB": "3B",
            "ip": "IP",
            "inningsPitched": "IP",
            "era": "ERA",
            "whip": "WHIP"
        }

        use_pitcher_order = (effective_stat_type == "pitching") if effective_stat_type != "both" else (block_is_pitching and not block_is_hitting)
        logical_list = PITCH_PREFERRED if use_pitcher_order else HIT_PREFERRED

        def find_actual_key(alts, statdict):
            for a in alts:
                if a in statdict:
                    return a
            return None

        final_keys = []
        for logical_name, alts in logical_list:
            actual = find_actual_key(alts, stat)
            if actual is not None and actual not in final_keys:
                final_keys.append(actual)
            else:
                final_keys.append("__missing__::" + logical_name)

        for key in final_keys:
            if key.startswith("__missing__::"):
                logical = key.split("::", 1)[1]
                label = key_labels.get(logical, logical)
                value = "-"
            else:
                label = key_labels.get(key, key)
                value = stat.get(key, "-")
            body_lines.append(f"{label}: {value}")
        body_lines.append("")

    # compose and set main_out HTML (preformatted)
    all_lines = header_lines + body_lines
    main_out.value = "<pre>" + "\n".join(all_lines) + "</pre>"

# UI layout and sequence boxes 
# Widget1 - logo + instruction
widget1_box = widgets.VBox([widgets.HTML("<h3>Widget 1 — Team Logo</h3>"), instruction1, text, out_logo])

# Widget2: roster selector + instruction (hidden initially)
instruction2 = widgets.HTML("<b>Choose an MLB team roster (does not have to be the same team you chose above):</b>")
select_options = []
for t in team_list:
    nm = t.get("name") or ""
    tid = t.get("id")
    roster_txt = get_team_roster_text(tid)  # caches roster text
    select_options.append((nm, roster_txt))

if not select_options:
    select_options = [("No teams available (network?)", "")]

# roster select with larger height so you can view many players without scrolling
roster_w = widgets.Select(
    options = select_options,
    value = None,
    description = 'Team roster:',
    layout = widgets.Layout(width = "60%", height = "500px")
)
widget2_inner = widgets.VBox([roster_w, teams_out, message_out])
widget2_box = widgets.VBox([widgets.HTML("<h3>Widget 2 — Rosters</h3>"), instruction2, widget2_inner])
widget2_box.layout.display = "none"  # hidden initially

# Widget3 contents
controls_row1 = widgets.HBox([team_dropdown, player_dropdown])
controls_row2 = widgets.HBox([season_slider])
buttons_row = widgets.HBox([show_roster_button, show_stats_button, refresh_teams_button])
widget3_box = widgets.VBox([widgets.HTML("<h3>Widget 3 — Player Stats</h3>"), controls_row1, controls_row2, buttons_row, main_out])
widget3_box.layout.display = "none"  # hidden initially

sequence_container = widgets.VBox([widget1_box, widget2_box, widget3_box])

# display the whole UI inside the single dashboard output (only this display call)
with dashboard_out:
    clear_output(wait = True)
    display(sequence_container)

display(dashboard_out)

# handlers for sequence and behavior

# Widget1: logo search behavior -> reveal widget2 when user types a team name
def _widget1_display_logo(change):
    if change.get("name") != "value":
        return
    key = (change.get("new") or "").strip().lower()
    url = image_map.get(key)
    if url:
        out_logo.value = f'<img src="{url}" width="150">'
    else:
        out_logo.value = ("<div style='color:#900'>Team not found.</div>" if key else "")
    # reveal widget2 if non-empty entry
    if key:
        widget2_box.layout.display = "flex"
    else:
        widget2_box.layout.display = "none"
        widget3_box.layout.display = "none"
        out_logo.value = ""
        main_out.value = "<pre></pre>"

text.observe(_widget1_display_logo, names = "value")

# Widget2: roster select behavior -> show roster text and reveal widget3
def _on_roster_select(change):
    if change.get("name") != "value":
        return
    try:
        idx = roster_w.index
        selected_label = roster_w.options[idx][0]
    except Exception:
        selected_label = None
    if selected_label:
        message_out.value = f"<div>The roster for the <b>{selected_label}</b> is:</div>"
    else:
        message_out.value = "<div>Selected roster:</div>"
    display_text = roster_w.value or "(no players)"
    # place roster text into main_out
    main_out.value = "<pre>" + display_text + "</pre>"
    # Reveal Widget3 now that a roster has been chosen
    widget3_box.layout.display = "flex"
    # If possible, set team_dropdown to the selected team
    if selected_label:
        tid = team_name_to_id.get(selected_label.lower())
        if tid is not None:
            try:
                team_dropdown.value = tid
            except Exception:
                pass

roster_w.observe(_on_roster_select, names = "value")

# Widget3 wiring: team -> populate players and show roster in main_out
def on_team_dropdown_change(change):
    if change.get("name") != "value":
        return
    team_id = change.get("new")
    # populate players and show roster
    try:
        roster = get_roster_for_team_id(team_id)
    except Exception as e:
        message_out.value = f"<div style='color:#900'>Error fetching roster: {e}</div>"
        player_dropdown.options = [("(choose a player)", None)]
        player_dropdown.value = None
        return
    players_on_team(team_id)
    display_roster(team_id)

team_dropdown.observe(on_team_dropdown_change, names = "value")

# when player selected show season + actions
def on_player_change(change):
    if change.get("name") != "value":
        return
    if player_dropdown.value:
        season_slider.layout.display = "flex"
        show_stats_button.disabled = False
    else:
        season_slider.layout.display = "none"
        show_stats_button.disabled = True

player_dropdown.observe(on_player_change, names = "value")

# show roster button
def on_show_roster(b):
    tid = team_dropdown.value
    display_roster(tid)

show_roster_button.on_click(on_show_roster)

# refresh teams button
def on_refresh(b):
    global team_list, player_dictionaries, team_name_to_id
    team_list = []
    team_name_to_id = {}
    player_dictionaries = {}
    try:
        r = sess.get(Teams_URL, timeout = 10)
        r.raise_for_status()
        team_list = r.json().get("teams", []) or []
        for t in team_list:
            nm = t.get("name")
            if nm:
                team_name_to_id[nm.lower()] = t.get("id")
    except Exception as e:
        message_out.value = f"<div style='color:#900'>Failed to refresh teams: {e}</div>"
        return
    team_dropdown.options = [("(choose a team)", None)] + [(t["name"], t["id"]) for t in team_list]
    team_dropdown.value = None
    player_dropdown.options = [("(choose a player)", None)]
    message_out.value = "<div>Teams refreshed. Choose a team to continue.</div>"

refresh_teams_button.on_click(on_refresh)

# show stats action
def on_show_stats(b):
    display_player_stats(player_dropdown.value, season_slider.value)

show_stats_button.on_click(on_show_stats)

# initial visibility & messages
season_slider.layout.display = "none"
show_stats_button.disabled = True

Output(layout=Layout(border_bottom='1px solid #444', border_left='1px solid #444', border_right='1px solid #44…

Borrowed Code Log:
Lines 441–503
Source: https://chatgpt.com/share/692368df-d14c-8013-aba1-3607731acb24
Description:
From this ChatGPT session, I learned how to construct a preferred pitching-stats list in the exact order I wanted. I also learned how to separate hitter and pitcher stats so that only relevant values appear depending on the player type. This helped me customize which stats were displayed in the output widget—showing important stats at the top and removing irrelevant ones. I also learned how to add homeruns and adjust hitting output to avoid clutter.

Lines 357–373
Source: https://chatgpt.com/share/6923615a-a19c-8013-9abb-832dc312f233
Description:
I learned how to combine multiple stat lines for a player who played for more than one team in the same season. Instead of returning separate lines (e.g., Blue Jays 25 games + Red Sox 50 games), the solution merges the stats into a single unified season line. This fixed my duplicate-output problem and ensured accurate totals.

Lines 14–22
Source: https://chatgpt.com/share/6923e127-cab0-8013-894f-307f9dab5254
Description:
This link taught me about using requests.Session() to make API calls more efficiently. It also introduced caching techniques to avoid repeating identical .get() calls. As a result, my code became much faster and avoided unnecessary network requests.

Lines 26, 85, 100, 106, 115, 121, 129, 132, 251, 674
Source: https://chatgpt.com/share/6923e41f-cbc4-8013-b80f-44c4a323501d
Description:
I learned how to use timeouts and exception handling to deal with slow or unresponsive API calls. Timeouts give the user helpful feedback instead of freezing the notebook, and exception blocks prevent crashes by showing what went wrong.

Lines 75, 101, 116
Source: https://chatgpt.com/share/6923e7e5-1b50-8013-9aab-22f0cf2718d7
Description:
I learned how docstrings help document each function, making it easier to understand inputs, outputs, usage, and behavior. With many functions in the project, docstrings helped organize the code and improve readability and debugging.

Lines 84, 120, 131
Source: https://chatgpt.com/share/6923ea75-6378-8013-bbf8-354e15d20d40
Description:
I learned how .format() improves URL template organization. Instead of rebuilding full URLs repeatedly, I can insert the variables (like player_id) into a reusable base template. This makes the code cleaner, reduces mistakes, and simplifies URL management.

Lines 137, 539, 610
Source: https://chatgpt.com/share/6923ec2b-37ec-8013-97cf-f47b76cb08b6
Description:
I learned to use simple HTML tags, especially <b>...</b>, inside output widgets to format text. This allowed me to highlight player names, stat headers, and important information, making the dashboard visually clearer and easier to read than plain text.

Lines 238, 267, 361
Source: https://chatgpt.com/share/6923f410-db24-8013-9790-f6c3fb142cb8
Description:
I learned how isinstance() helps check the exact type of a value before using it. This prevented type-related crashes (such as calling .get on None) and ensured the code behaves correctly when API responses vary in structure.

Lines 286–289
Source: https://chatgpt.com/share/6923f5bf-d4d4-8013-8505-5c58d9053e2f
Description:
I learned how the & operator is used with sets to quickly find overlapping elements. This helped me detect whether a stat block contained pitching stats by checking its keys against my pitching-keys set. Using & simplified the logic and made the code much cleaner.

Lines 387, 412–419
Source: https://chatgpt.com/share/6923fa0b-ab60-8013-9302-31e74560ef69
Description:
I learned how Python sets ensure unique values and prevent duplicates. The .union() method let me combine multiple sets easily. This was helpful when merging or comparing stat keys between different data groups. Sets and .union() simplified the logic and kept the code clean.